# Boosting
부스팅(Boosting)이란 단순하고 약한 학습기(Weak Learner)들를 결합해서 보다 정확하고 강력한 학습기(Strong Learner)를 만드는 방식.  
정확도가 낮은 하나의 모델을 만들어 학습 시킨뒤, 그 모델의 예측 오류는 두 번째 모델이 보완한다. 이 두 모델을 합치면 처음보다는 정확한 모델이 만들어 진다. 합쳐진 모델의 예측 오류는 다음 모델에서 보완하여 계속 더하는 과정을 반복한다.

- 약한 학습기들은 앞 학습기가 만든 오류를 줄이는 방향으로 학습한다.
- gradient boosting
    - 처음 모델은 y를 예측 두번째 부터는 앞 모델이 만든 오류를 예측 그것을 앞 모델에 업데이트하면 오류를 줄일 수 있다.
    - 그 오류를 update할 때 뺄까 더할까를 gradient descent 방법을 쓴다. 미분해서 나오는 값의 음수를 취해서 적용. 
    - 학습률을 작게하면 update가 조금씩 크면 많이 하게 된다. 그래서 크게하면 학습데이터에 너무 맞아 과대적합 될 수 있다.

## GradientBoosting
- 개별 모델로 Decision Tree 를 사용한다. 
- depth가 깊지 않은 트리를 많이 연결해서 이전 트리의 오차를 보정해 나가는 방식으로 실행한다.
- 오차를 보정할 때 경사하강법(Gradient descent)을 사용한다.
- 얕은 트리를 많이 연결하여 각각의 트리가 데이터의 일부에 대해 예측을 잘 수행하도록 하고 그런 트리들이 모여 전체 성능을 높이는 것이 기본 아이디어.
- 분류와 회귀 둘다 지원하는 모델 (GradientBoostingClassification, GrandientBoostingRegressor)
- 훈련시간이 많이 걸리고, 트리기반 모델의 특성상 희소한 고차원 데이터에서는 성능이 않좋은 단점이 있다.

### 주요 파라미터
- Decision Tree 의 가지치기 관련 매개변수
    - 각각의 tree가 복잡한 모델이 되지 않도록 한다. 
- learning rate
    - 이전 tree의 오차를 얼마나 강하게 보정할 것인지 제어하는 값. 
    - 값이 크면 보정을 강하게 하여 복잡한 모델을 만든다. 학습데이터의 정확도는 올라가지만 과대적합이 날 수있다. 
    - 값을 작게 잡으면 보정을 약하게 하여 모델의 복잡도를 줄인다. 과대적합을 줄일 수 있지만 성능 자체가 낮아질 수있다.
    - 기본값 : 0.1
- n_estimators
    - tree의 개수 지정. 많을 수록 복잡한 모델이 된다.
- n_iter_no_change, validation_fraction
    - validation_fraction에 지정한 비율만큼 n_iter_no_change에 지정한 반복 횟수동안 검증점수가 좋아 지지 않으면 훈련을 조기 종료한다.

- 보통 max_depth를 낮춰 개별 트리의 복잡도를 낮춘다. (5가 넘지 않게) 그리고 n_estimators를 가용시간, 메모리 한도에 맞춘뒤 적절한 learning_rate을 찾는다.




In [1]:
%%writefile util.py

# 평가지표 출력 함수
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def print_metrics(y, pred, title=None):
    acc = accuracy_score(y, pred)
    recall = recall_score(y, pred)
    precision = precision_score(y,pred)
    f1 = f1_score(y, pred)
    
    if title:
        print(title)
    print(f'정확도: {acc}, recall: {recall}, precision: {precision}, f1점수: {f1} ')

Overwriting util.py


In [2]:
from util import print_metrics
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [3]:
data = load_breast_cancer()
X, y = data['data'], data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)
X_train.shape, X_test.shape

((426, 30), (143, 30))

In [4]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=0)
gb.fit(X_train, y_train)
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

print_metrics(y_train, pred_train, title='Train')
print_metrics(y_test, pred_test, title='Test')

Train
정확도: 1.0, recall: 1.0, precision: 1.0, f1점수: 1.0 
Test
정확도: 0.958041958041958, recall: 0.9555555555555556, precision: 0.9772727272727273, f1점수: 0.9662921348314608 


In [9]:
# learning rate 변화
gb = GradientBoostingClassifier(random_state=0, max_depth=1, learning_rate=0.001, n_estimators=1000) # learning rate 기본값 : 0.1 -> 0.001(과소적합)
gb.fit(X_train, y_train)
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

print_metrics(y_train, pred_train, title='Train')
print_metrics(y_test, pred_test, title='Test')

Train
정확도: 0.9413145539906104, recall: 0.9887640449438202, precision: 0.9230769230769231, f1점수: 0.9547920433996384 
Test
정확도: 0.916083916083916, recall: 0.9888888888888889, precision: 0.89, f1점수: 0.9368421052631579 


In [32]:
# Feature 중요도를 조회
gb.feature_importances_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.17400003, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.07650318, 0.        , 0.36502689, 0.08764769, 0.        ,
       0.        , 0.        , 0.29682221, 0.        , 0.        ])

In [33]:
import pandas as pd
fi = pd.Series(gb.feature_importances_, index=data['feature_names'])
fi.sort_values(ascending=False)

worst perimeter            0.365027
worst concave points       0.296822
mean concave points        0.174000
worst area                 0.087648
worst radius               0.076503
mean radius                0.000000
concavity error            0.000000
worst symmetry             0.000000
worst concavity            0.000000
worst compactness          0.000000
worst smoothness           0.000000
worst texture              0.000000
fractal dimension error    0.000000
symmetry error             0.000000
concave points error       0.000000
compactness error          0.000000
mean texture               0.000000
smoothness error           0.000000
area error                 0.000000
perimeter error            0.000000
texture error              0.000000
radius error               0.000000
mean fractal dimension     0.000000
mean symmetry              0.000000
mean concavity             0.000000
mean compactness           0.000000
mean smoothness            0.000000
mean area                  0

### GridSearchCV 이용해 최적의 하이퍼파라미터 찾기

In [11]:
from sklearn.model_selection import RandomizedSearchCV

param = {
    'n_estimators':range(500,1001,100),
    'learning_rate':[0.001,0.05,0.01,0.1,0.5],
    'max_depth':[1,2,3],
    'subsample':[0.5,0.7,0.9,1]
}

rs = RandomizedSearchCV(GradientBoostingClassifier(random_state=0),
                       param,
                       cv=4,
                       n_iter=60,
                       scoring='accuracy',
                       n_jobs=-1)

In [13]:
rs.fit(X_train, y_train)

RandomizedSearchCV(cv=4, estimator=GradientBoostingClassifier(random_state=0),
                   n_iter=60, n_jobs=-1,
                   param_distributions={'learning_rate': [0.001, 0.05, 0.01,
                                                          0.1, 0.5],
                                        'max_depth': [1, 2, 3],
                                        'n_estimators': range(500, 1001, 100),
                                        'subsample': [0.5, 0.7, 0.9, 1]},
                   scoring='accuracy')

In [23]:
import pandas as pd
import numpy as np

In [15]:
result_df = pd.DataFrame(rs.cv_results_)

In [34]:
result_df.sort_values('rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
37,1.140187,0.073403,0.001995,2.665601e-07,0.7,500,3,0.1,"{'subsample': 0.7, 'n_estimators': 500, 'max_d...",0.962617,0.971963,0.981132,0.981132,0.974211,0.007669,1
58,0.621276,0.008443,0.000997,1.976862e-07,0.5,500,2,0.5,"{'subsample': 0.5, 'n_estimators': 500, 'max_d...",0.943925,0.971963,0.981132,0.990566,0.971896,0.017437,2
40,1.203837,0.033598,0.002244,4.324307e-04,0.5,700,3,0.1,"{'subsample': 0.5, 'n_estimators': 700, 'max_d...",0.953271,0.971963,0.981132,0.981132,0.971874,0.011374,3
24,1.933296,0.039266,0.002743,4.322933e-04,0.7,900,3,0.05,"{'subsample': 0.7, 'n_estimators': 900, 'max_d...",0.943925,0.971963,0.981132,0.981132,0.969538,0.015254,4
22,1.384613,0.008534,0.001745,4.317769e-04,0.7,900,2,0.1,"{'subsample': 0.7, 'n_estimators': 900, 'max_d...",0.943925,0.971963,0.981132,0.981132,0.969538,0.015254,4


In [17]:
rs.best_params_

{'subsample': 0.7, 'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.1}

In [18]:
rs.best_score_

0.9742108975489332

In [20]:
best_model = rs.best_estimator_
pred_test = best_model.predict(X_test)

In [21]:
print_metrics(y_test, pred_test, "best_model")

best_model
정확도: 0.958041958041958, recall: 0.9666666666666667, precision: 0.9666666666666667, f1점수: 0.9666666666666667 


In [26]:
fi = pd.Series(best_model.feature_importances_, index=data['feature_names'])
np.round(fi.sort_values(ascending=False),2)

worst perimeter            0.43
mean concave points        0.16
worst concave points       0.10
worst radius               0.09
worst area                 0.06
worst concavity            0.03
worst texture              0.03
mean texture               0.02
area error                 0.02
worst smoothness           0.02
mean concavity             0.01
radius error               0.01
concavity error            0.00
perimeter error            0.00
texture error              0.00
mean compactness           0.00
mean radius                0.00
mean area                  0.00
worst compactness          0.00
mean perimeter             0.00
mean fractal dimension     0.00
fractal dimension error    0.00
symmetry error             0.00
compactness error          0.00
worst symmetry             0.00
concave points error       0.00
mean symmetry              0.00
worst fractal dimension    0.00
smoothness error           0.00
mean smoothness            0.00
dtype: float64

# XGBoost(Extra Gradient Boost)
- https://xgboost.readthedocs.io/
- Gradient Boost 알고리즘을 기반으로 개선해서 나온 모델.
- 캐글 경진대회에서 상위에 입상한 데이터 과학자들이 사용한 것을 알려저 유명해짐.
- Gradient Boost의 단점인 느린수행시간을 해결하고 과적합을 제어할 수 있는 규제를 제공하여 성능을 높임.
- 두가지 개발 방법
    - [Scikit-learn 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)
    - [파이썬 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training)
- 설치
``
conda install -y -c anaconda py-xgboost
``



In [27]:
# !conda install -y -c anaconda py-xgboost

## Scikit-learn 래퍼 XGBoost
- XGBoost를 Scikit-learn프레임워크와 연동할 수 있도록 개발됨.
- Scikit-learn의 Estimator들과 동일한 패턴으로 코드를 작성할 수 있다.
- GridSearchCV나 Pipeline 등 Scikit-learn이 제공하는 다양한 유틸리티들을 사용할 수 있다.
- XGBClassifier: 분류
- XGBRegressor : 회귀 

### 주요 매개변수
- learning_rate : 학습률, 보통 0.01 ~ 0.2 사이의 값 사용
- n_estimators : week tree 개수
- max_depth: 트리의 depth 지정.

In [28]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=2, ramdom_state=0)

In [29]:
xgb.fit(X_train, y_train)

C:\Users\JeungDoYoung\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:29:56] WARNING: ..\src\learner.cc:541: 
Parameters: { ramdom_state } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:29:56] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=12, num_parallel_tree=1, ramdom_state=0,
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [30]:
pred_train = xgb.predict(X_train)
pred_test = xgb.predict(X_test)

In [31]:
print_metrics(y_train, pred_train, "XGB Train")
print_metrics(y_test, pred_test, "XGB Test")

XGB Train
정확도: 0.9953051643192489, recall: 0.9962546816479401, precision: 0.9962546816479401, f1점수: 0.9962546816479401 
XGB Test
정확도: 0.951048951048951, recall: 0.9555555555555556, precision: 0.9662921348314607, f1점수: 0.9608938547486034 
